# Init

In [1]:
%matplotlib inline
#%matplotlib notebook
import numpy as np
import pandas as pd
import math
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns

In [ ]:
from HigherOrderPathGenerator import CrossValidation_HigherOrderPathGenerator
from Embedding import HON_DeepWalk_Embedding, HONEM_Embedding, HON_NetMF_Embedding, HON_GraRep_Embedding, HON_Transition_Hierarchical_Embedding
from SyntheticNetworks import create_lattice_2nd_order_dynamic
from Visualizations import Visualization, EmbeddingView, Lattice2D_EmbeddingView

In [ ]:
size = 10
omega = 0.5
latgen = create_lattice_2nd_order_dynamic(size, omega, lattice_sep='-', check=True)

In [ ]:
for delta,f in latgen.creator.neighbor_funcs.items():
    print(delta,f.__name__)

In [ ]:
def disp_example_probabilities(example_indices):
    for ex in example_indices:
        source = latgen.source_paths[ex]
        name = 'FON' if len(source)==1 else ('HON ' + ('vertical' if source[0].split('-')[0]==source[1].split('-')[0] else 'horizontal'))
        probs = { next_node:prob for _,next_node,prob in latgen.transition_probs(source)}
        name += ', corner' if len(probs)==2 else ', border' if len(probs)==3 else ', interior'
        print(source, '->', probs, name)
disp_example_probabilities([0,1,10,11, 128,101,131,134, 102,100,132,133])
del disp_example_probabilities

In [ ]:
# This might be included into Lattice2D_EmbeddingView
def calc_edge_stats(edges: np.ndarray, title=None, plot: bool = True) -> pd.DataFrame:
    # assume edges.shape = (#edge pairs, embedding dimension)
    mean_edge = edges.mean(axis=0)
    mean_edge1 = mean_edge / np.linalg.norm(mean_edge)
    edges_len = np.linalg.norm(edges,axis=1)
    edges_spr = (edges @ mean_edge1) / edges_len
    edges_angle = np.arccos( edges_spr.clip(-1,1) )
    stats = pd.DataFrame({'len': edges_len, 'angle': edges_angle, 'angle360': edges_angle *180/math.pi})
    if plot:
        ax = stats.plot.scatter('len','angle', xlim=(0,max(edges_len)), ylim=(0,math.pi), c='#0000FF80')
        #ax = stats.plot.scatter('len','angle360', xlim=(0,max(edges_len)), ylim=(0,180))
        if title is not None:
            ax.figure.suptitle(title)
        ax.tick_params(labelright=True, right=True)
        ax.set_yticks(math.pi * np.linspace(0,1,num=5))
        ax.set_yticklabels(['0',r'$\frac{\pi}{4}$',r'$\frac{\pi}{2}$',r'$\frac{3\pi}{4}$',r'$\pi$'])
    return stats

# HONEM
* HONEM utilizes pairwise probabilities to describe how similar two nodes are
  * For simplicity, it just takes the output of BuildHON+
  * The neighborhood matrix defines a distance between two nodes, which is derived from aggregating transition probabilities.
However, these distances lack a theoretical justification like e.g. multistep transition probabilities
  * Moreover, GraRep argues against aggregating multistep transition probabilities
* Based on BuildHON+, which includes pruning of the rules for efficiency
  * Pruning rules should not affect the resulting probability model and therefore neither any down stream tasks
  * The effect of a small change to some estimated probabilities in BuildHON+ on the resulting model are still small (continuity). However, as soon as rule pruning kicks in, it may have a significant impact to the neighborhood matrix (**discontinuity**).

In [ ]:
emb_H = HONEM_Embedding(latgen, 128)
%time emb_H.train()
print('effective dimension', emb_H.dimension)

In [ ]:
def examine_HONEM_second_order_neighborhood(node):
    print('relevant transition probabilities:')
    keys = list(key for key in latgen.source_paths if len(key)==2 and key[0]==node)
    for key in keys:
        print(repr(key), '->', {next_node:prob for _,next_node,prob in latgen.transition_probs(key)})
    print('group these probabilities by their next_node:')
    n2_tmp = defaultdict(list)
    for key in keys:
        for _,next_node,prob in latgen.transition_probs(key):
            n2_tmp[next_node].append(prob)
    for n,a in sorted(n2_tmp.items()):
        print(' %s: %s' % (n,a))
    print('corresponding row of the 2nd order HONEM neighborhood matrix:')
    n2_row = emb_H.neighborhood_matrix(2).loc[emb_H.node2str((node,))]
    n2_row_nz = n2_row[n2_row!=0]
    for idx,dist in n2_row_nz.items():
        print(' %s: %f' % (idx,dist))
examine_HONEM_second_order_neighborhood('2-2')
del examine_HONEM_second_order_neighborhood

## T-SNE visualization of HONEM for the lattice with 2nd order dynamics
* Horizontal neighbors are closer than vertical ones (as designed)
* Needed to try different random_states for TSNE
* Instead tried to find a projection (based on determining averages for the embeddings of horizontal and vertical edges)
  * Average edge lengths in embedding space are indeed shorter for horizontal edges compared to vertical ones
  * However, the individual edges are almost perpendicular to their average, disproving the initial intuition of some simple manifold with mild curvature. (Surprisingly, the projection worked well.)

In [ ]:
ev_H = Lattice2D_EmbeddingView(emb_H, edge_distance=1)
vis_H = ev_H.visualize_TSNE(random_state=9, n_iter=1000)
vis_H.plot2(figsize=(9,4), dpi=400)
vis_H

In [ ]:
# for comparison
ev_H.visualize_lattice('Ground truth (FON)').plot2(figsize=(9,4), dpi=400)

In [ ]:
ev_H.visualize_proj(disp_lengths=True).plot2(figsize=(9,4), dpi=400)

In [ ]:
calc_edge_stats(ev_H.node_embedding_diff(latgen.creator.horizontal_edges1), 'Horizontal edges')
calc_edge_stats(ev_H.node_embedding_diff(latgen.creator.vertical_edges1), 'Vertical edges')
None

# Embedding HON Random Walks
## HON DeepWalk
* Combining HON random walks with Word2vec
* Horizontal neighbors are closer than vertical ones (as designed)
  * Projection-visualization works better than for HONEM (smaller angle)

In [ ]:
emb_D = HON_DeepWalk_Embedding(latgen, 128)
%time emb_D.train(window_size=10)
print('effective dimension', emb_D.dimension) # unchanged

In [ ]:
ev_D = Lattice2D_EmbeddingView(emb_D, edge_distance=1)
vis_D = ev_D.visualize_TSNE(random_state=6, n_iter=1000)
vis_D.plot2(figsize=(9,3), dpi=400)
vis_D

In [ ]:
ev_D.visualize_proj().plot2(figsize=(9,3), dpi=400)

In [ ]:
calc_edge_stats(ev_D.node_embedding_diff(latgen.creator.horizontal_edges1), 'Horizontal edges')
calc_edge_stats(ev_D.node_embedding_diff(latgen.creator.vertical_edges1), 'Vertical edges')
None

## HON NetMF (pairwise)
* Utilizing a breadth-first search to speed up calculation of PMI for large window_sizes.
* Approximates HON DeepWalk (negative=1 works better that e.g. negative=5)
* Horizontal neighbors are closer than vertical ones (as designed and expected from DeepWalk)

In [ ]:
emb_N_pairs = HON_NetMF_Embedding(latgen, 128, pairwise=True)
%time emb_N_pairs.train(window_size=10, negative=1, optimized=True)
print('effective dimension', emb_N_pairs.dimension)

In [ ]:
ev_N_pairs = Lattice2D_EmbeddingView(emb_N_pairs, use_source=True, edge_distance=1)
vis_N_pairs = ev_N_pairs.visualize_TSNE(random_state=19, n_iter=1000)
vis_N_pairs.plot2(figsize=(9,4), dpi=400)

In [ ]:
calc_edge_stats(ev_N_pairs.node_embedding_diff(latgen.creator.horizontal_edges1), 'Horizontal edges')
calc_edge_stats(ev_N_pairs.node_embedding_diff(latgen.creator.vertical_edges1), 'Vertical edges')
None

In [ ]:
ev_N_pairs.visualize_proj(disp_lengths=True).plot2(figsize=(9,4), dpi=400)

## HON NetMF (paths)
* Pairwise-NetMF embedds keys like '0-0' while Paths-NetMF embedds keys like ('0-0','1-0'), therefore the latter has to embedd more keys.
* Plots:
  1. if key is a pair of nodes, direction(key) contains the difference between their corresponding coordinates; otherwise it is 'none'. Example: for the key ('0-0','1-0') indicating 'moving to the right', its direction is '(1,0)' and (x_orig,y_orig) correspond to the **second** node of the key. See the third plot for details.
  2. Excluding all keys of length > 1 reveals a grid structure similar to DeepWalk and NetMF (pairwise): horizontal neighbors are closer than vertical ones
  3. This plot was rotated such that the horizontal direction of the lattice correcponds to the x-axis and the vertical direction to the negative y-axis.

In [ ]:
emb_N = HON_NetMF_Embedding(latgen, 128, pairwise=False)
%time emb_N.train(window_size=5, negative=5)
print('effective dimension', emb_N.dimension)

In [ ]:
ev_N = Lattice2D_EmbeddingView(emb_N, use_source=True, edge_distance=1)
vis_N = ev_N.visualize_TSNE(random_state=7, n_iter=2000)
vis_N.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

In [ ]:
print('Ignore embeddings of source-keys with length > 1 (i.e. only FON probabilities)')
vis_N.plot1(figsize=(6,6), dpi=400, filter_col='key_len', filter_values={1}, hue='x_orig')

In [ ]:
# use edges to display node hierarchy
#vis_N._edges = list( (ev_N.key2str(key), ev_N.key2str(ev_N.node2key(key[-1]))) for key in ev_N.keys if len(key)>1 )
#vis_N.config['edges'] = 'hack...'
# markers
dir_syms = { 'none': 'o', 'left': '<', 'right': '>', 'up': '^', 'down': 'v' }
dir_rank = { k:i for i,k in enumerate(dir_syms.keys()) }
vis_N._data['sort'] = vis_N._data['direction'].map(dir_rank)
vis_N._data.sort_values(['sort'], inplace=True)
#vis_N._data.sort_values(['direction'], inplace=True, key=lambda d:dir_rank[d]) # new in pandas version 1.1.0

vis_N.plot1(figsize=(7,7), dpi=400, style='direction', markers=list(dir_syms.values())) #, hue='x_orig')
#vis_N.save_describe('tsne_netmf-w5n5-mix_synth_dir.png', comment)

# Embedding HON Transition Probabilities (Random Walk of length1)
## HON NetMF for window_size=1
* Random walks of length 1 are just transition probabilities
* Split the nodes of the lattice into two groups (even and odd) based on the sum of their positions in the lattice.
  * 'even' nodes are esclusively connected to 'odd' nodes and vice-versa. Since positive probabilities implies that the scalar product of their embeddings should be bigger than the same for probability zero. 
  * Due to sharing common neighbors, 'even' nodes are tied together. The same holds for 'odd' ones.
  * Negative sampling pushes 'even' and 'odd' nodes away from each other. Hence, the scalar product of their embeddings should be strongly negative.
  * Therefore, the embedding splits into two clusters.
* This holds for both versions: pairs and paths.

In [ ]:
emb_N1 = HON_NetMF_Embedding(latgen, 128, pairwise=False)
%time emb_N1.train(window_size=1, negative=5)
emb_N1_pairs = HON_NetMF_Embedding(latgen, 128, pairwise=True)
%time emb_N1_pairs.train(window_size=1, negative=5)

In [ ]:
# Autotransform fails because the vertical edges point in opposite directions for the two groups (even and odd)
ev_N1_pairs = Lattice2D_EmbeddingView(emb_N1_pairs, use_source=True, edge_distance=2)
vis_N1_pairs = ev_N1_pairs.visualize_TSNE(random_state=12, n_iter=1000, autotransform=False)
vis_N1_pairs.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

In [ ]:
# embedding is split into two clusters: even and odd
vis_N1_pairs.plot1(hue='parity', figsize=(5,4))

In [ ]:
ev_N1 = Lattice2D_EmbeddingView(emb_N1, use_source=True, edge_distance=2)
vis_N1 = ev_N1.visualize_TSNE(random_state=6, n_iter=4000, autotransform=False) # random_state=2,6,19
vis_N1.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

In [ ]:
# check PMI for window_size=1
def check_pmi(start):
    start_str = emb_N1.path2str(start)
    pmi = emb_N1.PMI.loc[start_str] # transition = {'0-0':0.25, '1-1':0.25}
    print('pmi',(pmi[pmi!=0].sort_values()).to_dict())
    sd = pd.Series({ emb_N1.node2str(v):p for _,v,p in latgen.transition_probs(start=()) })
    pmi_mul_sd = (pmi*sd).sort_index()
    print('pmi * sd',(pmi_mul_sd[pmi_mul_sd!=0].sort_values()).to_dict())
    print('transition prob',{next_node:prob for _,next_node,prob in latgen.transition_probs(start)})
#check_pmi( ('0-0','1-0') )
check_pmi( ('5-5','6-5') )

In [ ]:
# Question: Is this structure an artifact due to the decision to use different embeddings for source and target?
# verify that PMI is symmetric (in fact this holds even for window_sizes > 1)
pmi = emb_N1_pairs.PMI
print(f'NetMF(pairwise={emb_N1_pairs._pairwise}).train(window_size={emb_N1_pairs._window_size}, ...)')
print('|PMI-PMI.T|<= %g' % np.abs(pmi.values - pmi.values.T).max())
# calculate Eigendecomposition
negative=1
mat = np.log((pmi.values / negative).clip(1))
w,v = np.linalg.eigh(mat)
print(f'#positive Eigenvalues={np.count_nonzero(w>0)}, #negative Eigenvalues={np.count_nonzero(w<0)}')
# => matrix has negative Eigenvalues
# and using same embeddings for source and target implies their product is non-negative definite

In [ ]:
### Experiment: Try factoring the log(PMI)-matrix using a symmetric factorization.
### It did not work...

#from Embedding import ABCSymmetricEmbedding
#class Symmetric_NetMF_Embedding(ABCSymmetricEmbedding):
#    def __init__(self, emb: HON_NetMF_Embedding, negative=1):
#        super().__init__(emb._dimension, emb.target_nodes)
#        self.node2str = emb.node2str
#        self._gen = emb._gen
#        pmi = emb.PMI.values
#        pmi = (pmi + pmi.T) / 2 # ensure pmi is symmetric
#        mat = np.log((pmi / negative).clip(1))
#        w,v = np.linalg.eigh(mat) # real valued eigendecomposition
#        w = np.real(w)
#        self.Eigenvalues = w
#        w_pos = w.clip(0)
#        self._embedding = v.T @ np.diag(w_pos**0.5)
#emb_N1_pairs_sym = Symmetric_NetMF_Embedding(emb_N1_pairs, 1)
#ev_N1_pairs_sym = Lattice2D_EmbeddingView(emb_N1_pairs_sym, edge_distance=1)
#vis_N1_pairs_sim = ev_N1_pairs_sym.visualize_TSNE(random_state=0, n_iter=4000)
#vis_N1_pairs_sim.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

## Embedding the transition matrix with SGD
* While NetMF uses (unweighted!) matrix factorization (SVD), also a stochastic gradient descent (SGD) variant was implemented, which solves a weighted matrix factorization problem.
* While tuning the learning rates is time consuming, SGD nevertheless has some benefits:
  * Better suitable for large networks (cf. [App])
  * Random walk based methods have memory and time requirements independent of size of neighborhood (cf. [GraphSAGE])
  * Ability to include penalty terms
  * Missing or uncertain values can be down-weighted to avoid any impact on the embedding. This can be useful to design some sort of cross-validation. In an unweighted procedure (such as SVD) there is no way to specify that some values should not impact the result.

In [ ]:
# Training the following model is slow. This will abort a "Run all".
#raise Exception()

In [ ]:
def plot_obj(title:str, data):
    # see Plots_ExpVis_synth.ipynb
    y = data['objectives']
    print('Last objective %f' % y[-1])
    total_steps = data['total_steps']
    steps = data['steps']
    previous_steps = total_steps - steps
    x = previous_steps + np.arange(len(y)) * steps / len(y)
    fig = plt.figure(figsize=(6,4),dpi=200)
    ax = fig.gca()
    y_pos = data['objectives_pos']
    y_neg = data['objectives_neg'] * data['negative']
    y_penalty = data['objectives_penalty'] * data['penalty']
    ax.plot(x,data['objectives'], color='black',label='Loss')
    ax.fill_between(x,0,y_pos,label='Positive')
    ax.fill_between(x,y_pos,y_pos+y_neg,label='Negative')
    if data['penalty']>0:
        ax.fill_between(x,y_pos+y_neg,y_pos+y_neg+y_penalty,label='Penalty')
    fig.suptitle(title)
    ax.set_xlabel('steps')
    ax.set_ylabel('loss') # 'objective'
    ax.legend(loc='upper right', ncol=1)

In [ ]:
emb_T = HON_Transition_Hierarchical_Embedding(latgen, dimension=128, seed=1)
learning_rates = dict(learning_rate_start=0.03, learning_rate_end=0.02)
# adding max_path_len=1 speeds calculation up further
%time obj = emb_T.train(steps=300, negative=5, penalty=0, debug_objective=10, **learning_rates)
plot_obj('HON_Transition_Hierarchical', obj)

In [ ]:
learning_rates = dict(learning_rate_start=0.005, learning_rate_end=0.001)
%time obj = emb_T.train(steps=1000, negative=5, penalty=0, debug_objective=10, **learning_rates)
plot_obj('HON_Transition_Hierarchical (cont.)', obj)

In [ ]:
#learning_rates = dict(learning_rate_start=0.001, learning_rate_end=0.001)
#%time obj = emb_T.train(steps=2000, negative=5, penalty=0, debug_objective=100, **learning_rates)
#plot_obj('HON_Transition_Hierarchical (cont.)', obj)

In [ ]:
ev_T = Lattice2D_EmbeddingView(emb_T, use_source=True, edge_distance=2)
vis_T = ev_T.visualize_TSNE(random_state=18, n_iter=4000)
vis_T.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

## Penalizing...
* When embedding paths instead of single nodes, we already observed, that paths which have their last node in common are close together.
  * Speed up optimization by first findng embeddings for storter paths ans using them as starting values for longer paths.
  * Rules starting with a longer path have less support, pulling their embedding towards the one of a similar but shorter path might stabilize the procedure. This is implemented via penalty.

## GraRep
GraRep is implemented similarly to NetMF. While the k-step transition probabilities are embedded separately in GraRep, they are averaged in NetMF.

In [ ]:
emb_G = HON_GraRep_Embedding(latgen, emb_N._dimension, emb_N._window_size, emb_N._pairwise, neg_stationary=True)
%time emb_G.train(emb_N._negative)

In [ ]:
#check consistency between PMI calculation of NetMF and GraRep
np.abs(emb_N._PMI - emb_G._PMI.mean(axis=2)).max()

In [ ]:
ev_G = Lattice2D_EmbeddingView(emb_G, use_source=True, edge_distance=2)
vis_G = ev_G.visualize_TSNE(random_state=12, n_iter=1000, autotransform=False)
vis_G.plot2(figsize=(9,4), dpi=400, style='direction', alpha=0.5)

In [ ]:
if False:
    ev = ev_N1
    for x in range(20):
        vis = ev.visualize_TSNE(random_state=x, n_iter=1000)
        vis.plot2(figsize=(9,4), dpi=400)